In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# input dataset
mnist = input_data.read_data_sets('MNIST_data/', one_hot =True)


W0830 21:09:21.152775  8864 deprecation.py:323] From <ipython-input-2-ff7a233c4436>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0830 21:09:21.156773  8864 deprecation.py:323] From H:\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0830 21:09:21.157775  8864 deprecation.py:323] From H:\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implemen

Extracting MNIST_data/train-images-idx3-ubyte.gz


W0830 21:09:21.388699  8864 deprecation.py:323] From H:\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0830 21:09:21.390699  8864 deprecation.py:323] From H:\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0830 21:09:21.434685  8864 deprecation.py:323] From H:\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Plea

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# input image 28*28
n_inputs = 28 # 输入一行，一行有28个数据
max_time = 28 # 一共28行
lstm_size = 100 # 隐层单元个数
n_classes = 10 # 分类个数 10 
batch_size = 50 # 每批次50样本
n_batch = mnist.train.num_examples // batch_size # 计算共多少批次

# 这里none表示第一维度可以是任意的长度
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None,10])

# initialize weights 
weights = tf.Variable(tf.truncated_normal([lstm_size, n_classes], stddev=0.1))

biases = tf.Variable(tf.constant(0.1, shape=[n_classes]))


# 定义RNN网络
def RNN(X, weights, biases):
    # input = [batch_size, max_time, n_inputs]
    inputs = tf.reshape(X, [-1,max_time, n_inputs])
    # 定义LSTM基本单元
    lstm_cell = tf.contrib.rnn.LSTMBlockCell(lstm_size)
#      tf.nn.rnn_cell.LSTMCell   tf.contrib.rnn.LSTMBlockCell
#     lstm_cell = tf.contrib.rnn.core_rnn_cell.BasicLSTMCell(lstm_cell, inputs, dtype=tf.float32)
    # final_state [0] 是cell state
    # final_state [1] 是hidden_state
    outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, inputs, dtype=tf.float32)
    results = tf.nn.softmax(tf.matmul(final_state[1], weights) + biases)
    return results

# 计算RNN返回结果
prediction = RNN(x, weights, biases)
# loss function 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
# 使用AdamOptimizer优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# 结果保存在一个布尔型列表中
correct_prediction = tf.equal(tf.math.argmax(y, 1), tf.math.argmax(prediction,1)) # argmax返回一维张量中最大的值所在的位置
# 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) # 吧correct_prediction 变为float32类型
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(6):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict = {x:batch_xs, y:batch_ys})
            
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print('Iter:' + str(epoch) + ', Testing Accuracy=' + str(acc))


W0830 21:09:21.682605  8864 deprecation.py:323] From <ipython-input-3-bdcb42406d20>:29: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0830 21:09:21.746585  8864 deprecation.py:506] From H:\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0830 21:09:21.959534  8864 deprecation.py:323] From <ipython-input-3-bdcb42406d20>:36: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backpr

Iter:0, Testing Accuracy=0.6885
Iter:1, Testing Accuracy=0.8076
Iter:2, Testing Accuracy=0.9091
Iter:3, Testing Accuracy=0.9163
Iter:4, Testing Accuracy=0.9316
Iter:5, Testing Accuracy=0.9379
